# Importing Libraries

In [ ]:
!pip install -U efficientnet
import sys
import os
from os.path import join
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import efficientnet.keras as efn 
import tensorflow as tf
import tensorflow_addons as tf
from keras.preprocessing.image import load_img, img_to_array
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization
from keras.utils import np_utils #, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

# Reading Dataset

In [ ]:
PATH = "../input/melanoma/dermmel/DermMel/"

In [ ]:
print('Total Directories')
display(os.listdir(PATH))

In [ ]:
# Checkcking content of the directories

PATHtrain = PATH + 'train_sep/'
print(len(os.listdir(PATHtrain)), " TRAIN Directories of photos")
drs = os.listdir(PATHtrain)
im_len = 0
for label in sorted(drs):
    print(label,len(os.listdir(PATHtrain + label +'/')))
    im_len = im_len + len(os.listdir(PATHtrain + label +'/'))

print("Total TRAIN photos ", im_len)
print("_"*50)

PATHvalid = PATH + 'valid/'
print(len(os.listdir(PATHvalid)), " VALID Directories of photos")
drs = os.listdir(PATHvalid)
im_len = 0
for label in sorted(drs):
    print(label,len(os.listdir(PATHvalid + label +'/')))
    im_len = im_len + len(os.listdir(PATHvalid + label +'/'))

print("Total Validation photos ", im_len)
print("_"*50)

PATHtest = PATH + 'test/'
print(len(os.listdir(PATHtest)), " TEST Directories of photos")
drs = os.listdir(PATHtest)
im_len = 0

for label in sorted(drs):
    print(label,len(os.listdir(PATHtest + label +'/')))
    im_len = im_len + len(os.listdir(PATHtest + label +'/'))

print("Total Testing photos ", im_len)
print("_"*50)

In [ ]:
# Check the photos and their labels 

TestNum = 40
diag1 = 'Melanoma'
diag2 = 'NotMelanoma'

# MELANOMA TRAIN
image_dir = PATHtrain +'/'+diag1+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=(224, 224))
imgplot = plt.imshow(img)
print("TRAIN ",diag1," photo number ", TestNum)
plt.show()

# NOTMELANOMA TRAIN
image_dir = PATHtrain +'/'+diag2+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=(224, 224))
imgplot = plt.imshow(img)
print("TRAIN ",diag2," photo number ", TestNum)
plt.show()


#MELANOMA VALID
image_dir = PATHvalid +'/'+diag1+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=(224, 224))
imgplot = plt.imshow(img)
print("VALID ",diag1," photo number ", TestNum)
plt.show()

#NOTMELANOMA VALID
image_dir = PATHvalid +'/'+diag2+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=(224, 224))
imgplot = plt.imshow(img)
print("VALID ",diag2," photo number ", TestNum)
plt.show()


#MELANOMA TEST
image_dir = PATHtest +'/'+diag1+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=(224, 224))
imgplot = plt.imshow(img)
print("TEST ",diag1," photo number ", TestNum)
plt.show()

#NOTMELANOMA TEST
image_dir = PATHtest +'/'+diag2+'/'
img_name = os.listdir(image_dir)[TestNum]
img_path = image_dir+str(img_name)
img = image.load_img(img_path, target_size=(224, 224))
imgplot = plt.imshow(img)
print("TEST ",diag2," photo number ", TestNum)
plt.show()


# Modelling

In [ ]:
from efficientnet.tfkeras import EfficientNetB7
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
from keras.regularizers import l2

In [ ]:
eff = EfficientNetB7(input_shape=(224 , 224 , 3) , include_top=False , weights='imagenet')

model = Sequential()
model.add(eff)
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128 , activation='relu' , kernel_regularizer=regularizers.l2(0.001) ))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(2, activation='sigmoid')) 

In [ ]:
earlystopping = EarlyStopping(monitor ="val_loss", 
                                        patience = 8, 
                                        restore_best_weights = True)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'] )  

In [ ]:
model.summary()

In [ ]:
# Prep the Train /  Valid and Test directories for the generator

train_dir = PATHtrain
validation_dir = PATHvalid
test_dir = PATHtest
batch_size = 20
target_size=(224, 224)

#train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,target_size=target_size,batch_size=batch_size)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,target_size=target_size,batch_size=batch_size)
test_generator = test_datagen.flow_from_directory(
    test_dir,target_size=target_size,batch_size=batch_size)

In [ ]:
print(train_generator.class_indices)

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=3,
                              steps_per_epoch = 10682 // batch_size,
                              validation_data = validation_generator,
                              validation_steps = 3562 // batch_size ,
                              callbacks=[earlystopping])


# Evaluation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.save_weights('model.hdf5')
#model.load_weights('best_model.hdf5')

In [ ]:
y=np.concatenate([test_generator.next()[1] for i in range(test_generator.__len__())])
true_labels=np.argmax(y, axis=-1)

prediction= model.predict(test_generator, verbose=2)
prediction=np.argmax(prediction, axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(true_labels,prediction) 
print('Accuracy: %.3f' % acc)
from sklearn.metrics import precision_score
precision = precision_score(true_labels,prediction,labels=[1,2], average='micro')
print('Precision: %.3f' % precision)
from sklearn.metrics import recall_score
recall = recall_score(true_labels,prediction, average='binary')
print('Recall: %.3f' % recall)
from sklearn.metrics import f1_score
score = f1_score(true_labels,prediction, average='binary')
print('F-Measure: %.3f' % score)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=true_labels, y_pred=prediction)
cm
#